#  Assignment 4 - Text Analysis
by Amitay Biton, January 2022

### Text Classification by Gender


In [1]:
# IMPORT MODULES 
import pandas as pd
import hebrew_tokenizer as ht
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import confusion_matrix

c:\Users\amita\Desktop\school\ML\Task4\MLGenderPrediction


C:\Users\amita\AppData\Local\Programs\Python\Python39\lib\site-packages\hebrew_tokenizer\tokenizer.py:121: FutureWarning: Possible nested set at position 843
  self.scanner = re.compile(


## Stage 1: Init
#### Declaring & Implementing Side Functions :

In [2]:
def load_dataset(csv_path):
    return pd.read_csv(csv_path)

In [3]:
def clean_story(story):
    tokens = ht.tokenize(story)
    words = []
    for token in tokens:
        if token[0] == 'HEBREW':
            words.append(token[1])
    return words
    # return " ".join(words)

In [4]:
def collect_features_form_story(story):
    # female feature lookup:
    f1 = 0
    for i in range(1, len(story)):
        if (story[i-1].endswith("אני") and (story[i].endswith("ת") or story[i].endswith("ה"))):
            f1 += 1
    # -----------------------------
    f2 = 0
    for i in range(len(story)):
        if (story[i] == "בעלי" or story[i] == "ארוסי" or (story[i-1] == "בן" and story[i] == "זוגי")):
            f2 += 1
    # -----------------------------
    f3 = 0
    for i in range(len(story)):
        if  (story[i-1].endswith("הייתי") and story[i] == "בת"):
            f3 += 1
    # -----------------------------
    f4 = 0
    for i in range(len(story)):
        if  (story[i-1].endswith("הייתי") and story[i] == "ילדה"):
            f4 += 1
    # -----------------------------
    f5 = 0
    for i in range(1, len(story)):
        if (story[i-2].endswith("אני") and (story[i-1] == "לא") and (story[i].endswith("ת") or story[i].endswith("ה"))):
            f5 += 1
    # -----------------------------
    f6 = 0
    for i in range(1, len(story)):
        if (story[i-1].endswith("אנחנו") and story[i].endswith("ות")):
            f6 += 1
    # -----------------------------
    f7 = 0
    for i in range(1, len(story)):
        if (story[i-2].endswith("אנחנו") and (story[i-1] == "לא") and story[i].endswith("ות")):
            f7 += 1

    # male feature lookup:
    m1 = 0
    for i in range(len(story)):
        if (story[i] == "אישתי" or story[i] == "ארוסתי" or (story[i-1] == "בת" and story[i] == "זוגתי")):
            m1 += 1
    # -----------------------------
    m2 = 0
    for i in range(len(story)):
        if  (story[i-1].endswith("הייתי") and story[i] == "בן"):
            m2 += 1
    # -----------------------------
    m3 = 0
    for i in range(len(story)):
        if  (story[i-1].endswith("הייתי") and story[i] == "ילד"):
            m3 += 1
    # -----------------------------
    m4 = 0
    for i in range(1, len(story)):
        if (story[i-1].endswith("אנחנו") and story[i].endswith("ים")):
            m4 += 1
    # -----------------------------
    m5 = 0
    for i in range(1, len(story)):
        if (story[i-2].endswith("אנחנו") and (story[i-1] == "לא") and story[i].endswith("ים")):
            m5 += 1
    # -----------------------------
    unique = len(set(story))
    length = len(story)
    return [f1,f2,f3,f4,f5,f6,f7,m1,m2,m3,m4,m5,unique,length]

In [5]:
def create_df_of_feature(df):
    feature_names = ["story","f1", "f2", "f3", "f4", "f5", "f6", "f7", "m1", "m2", "m3", "m4", "m5", "unique", "length","gender"]
    keys = []
    for row in df.values:
        words = clean_story(row[0])
        if row[1] == "f":
            gender = 0 
        else: 
            gender = 1
        vector = [" ".join(words)] + collect_features_form_story(words) + [gender]
        keys.append(vector)
    return pd.DataFrame(data=keys,columns=feature_names)


In [6]:
def score_ass(y_pred, y_true):
    return (f1_score(y_true, y_pred, pos_label=0) + f1_score(y_true, y_pred, pos_label=1))/2

In [7]:
def add_tf_idf_featues(vectors, X):
    rows = []
    for i in range(X.shape[0]):
        rows.append(vectors[i].tolist()[0] + list(X.drop(['story'],axis=1).iloc[i]))
    return pd.DataFrame(data=rows)

In [8]:
def create_df_of_feature_for_test(df):
    feature_names = ["story","f1", "f2", "f3", "f4", "f5", "f6", "f7", "m1", "m2", "m3", "m4", "m5", "unique", "length"]
    keys = []
    for row in df.values:
        words = clean_story(row[1])
        vector = [" ".join(words)] + collect_features_form_story(words)
        keys.append(vector)
    return pd.DataFrame(data=keys,columns=feature_names)


## stage 2: Data Cleaning and Feature Extraction
At first we load the 'annotated_corpus_for_train.csv' dataset, split to Train and Test Groups, and extract features in basic way


In [9]:
df = load_dataset('./data/annotated_corpus_for_train.csv')

In [10]:
new_df = create_df_of_feature(df)
X = new_df.drop(['gender','story'],axis=1)
y = new_df['gender']


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,f1,f2,f3,f4,f5,f6,f7,m1,m2,m3,m4,m5,unique,length
586,3,0,0,0,0,0,0,0,0,0,0,0,253,317
131,0,0,0,0,0,0,0,0,0,0,0,0,256,353
44,0,0,0,0,0,0,0,0,0,0,0,0,164,200
70,0,1,0,0,0,0,0,0,0,0,2,0,266,355
581,0,0,0,0,0,0,0,0,0,0,1,0,356,581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0,0,0,0,0,0,0,4,0,0,1,0,302,397
106,1,0,0,0,0,0,0,0,0,0,0,0,219,302
270,0,0,0,0,0,0,0,0,0,0,0,0,328,431
435,0,2,0,0,0,0,0,0,0,0,0,0,242,298


## Stage 3: Building Model, Training and Assessing its Predicions
After building a feature vector, we will create a model and assassing the predicions

In [12]:
cw = {0 :3.23, 1:1}
clf = LogisticRegression(random_state=42, class_weight=cw ,max_iter=1000).fit(X_train, y_train)

print(clf.score(X_test, y_test))

print("confusion_matrix(y_test, clf.predict(test1)) :")
print(confusion_matrix(y_test, clf.predict(X_test)))

print("confusion_matrix(y_train, clf.predict(train1)) :")
print(confusion_matrix(y_train, clf.predict(X_train)))

print("score_ass(clf.predict(test1), y_test) :")
print(score_ass(clf.predict(X_test), y_test))


0.7682119205298014
confusion_matrix(y_test, clf.predict(test1)) :
[[23 14]
 [21 93]]
confusion_matrix(y_train, clf.predict(train1)) :
[[ 80  61]
 [100 361]]
score_ass(clf.predict(test1), y_test) :
0.7047650969219597


# Stage 4: TF-IDF Vectorization
after evaluating the model, now we will try to build a better feature vector in order to predict better.

In [13]:
vectorizer = TfidfVectorizer(max_df=1.,min_df=0.05,ngram_range = (1,2))
X = new_df.drop(['gender'],axis=1)
y = new_df['gender']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectors_train = vectorizer.fit_transform(X_train['story'])
vectors_test = vectorizer.transform(X_test['story'])

### feature selection:

In [14]:
selector = VarianceThreshold(threshold=0.001)
vectors_train = selector.fit_transform(vectors_train)
vectors_test = selector.transform(vectors_test)

fs = SelectKBest(chi2, k=50)
vectors_train_fs = fs.fit_transform(vectors_train, y_train).todense()
vectors_test_fs = fs.transform(vectors_test).todense()


In [15]:
X_train = add_tf_idf_featues(vectors_train_fs,X_train)
X_test = add_tf_idf_featues(vectors_test_fs, X_test)
X_train

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.000000,0.0,0.000000,0.088340,0.124166,0.0,0.229207,0.073776,0.000000,0.000000,...,0,0,0,0,0,0,0,0,253,317
1,0.030174,0.0,0.000000,0.038239,0.000000,0.0,0.049607,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,256,353
2,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,164,200
3,0.055107,0.0,0.000000,0.000000,0.000000,0.0,0.045300,0.000000,0.218809,0.000000,...,0,0,0,0,0,0,2,0,266,355
4,0.151557,0.0,0.030876,0.000000,0.000000,0.0,0.015573,0.120302,0.000000,0.000000,...,0,0,0,0,0,0,1,0,356,581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,0.028493,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0,0,0,4,0,0,1,0,302,397
598,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.110719,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,219,302
599,0.000000,0.0,0.050701,0.039424,0.000000,0.0,0.051145,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,328,431
600,0.000000,0.0,0.060617,0.047135,0.000000,0.0,0.091722,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,242,298


### Revaluation
Evaluation of the model with the TF_IDF vector included in the feature vector

In [16]:
cw = {0 :2.1, 1:1}
clf = LogisticRegression(random_state=42, class_weight=cw ,max_iter=1000).fit(X_train, y_train)

print(clf.score(X_test, y_test))

print("confusion_matrix(y_test, clf.predict(X_test)) :")
print(confusion_matrix(y_test, clf.predict(X_test)))

print("confusion_matrix(y_train, clf.predict(X_train)) :")
print(confusion_matrix(y_train, clf.predict(X_train)))

print("score_ass(clf.predict(X_test), y_test) :")
print(score_ass(clf.predict(X_test), y_test))

0.8344370860927153
confusion_matrix(y_test, clf.predict(X_test)) :
[[ 21  16]
 [  9 105]]
confusion_matrix(y_train, clf.predict(X_train)) :
[[ 82  59]
 [ 31 430]]
score_ass(clf.predict(X_test), y_test) :
0.7602413464591933


# Stage 5: Retraning
Now, we will train ower model on all of our train dataset

In [17]:
new_df = create_df_of_feature(df)
X_train = new_df.drop(['gender'],axis=1)
y_train = new_df['gender']
vectors_train = vectorizer.fit_transform(X_train['story'])

selector = VarianceThreshold(threshold=0.001)
vectors_train = selector.fit_transform(vectors_train)

fs = SelectKBest(chi2, k=50)
vectors_train_fs = fs.fit_transform(vectors_train, y_train).todense()

X_train = add_tf_idf_featues(vectors_train_fs,X_train)


# Stage 6: Predicting Result
Now, after training the model, we will load 'corpus_for_test.csv' dataset, extract features, vectorize, and predict the gender using our trained model.
Result will be exported to 'classification_results.csv'

In [18]:
test_df = load_dataset("data/corpus_for_test.csv")

In [19]:
new_df_test = create_df_of_feature_for_test(test_df)
vectors_test = vectorizer.transform(new_df_test['story'])
vectors_test = selector.transform(vectors_test)
vectors_test_fs = fs.transform(vectors_test).todense()


X_test = add_tf_idf_featues(vectors_test_fs,new_df_test)
X_test

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.033931,0.000000,0.0,0.086731,0.000000,0.080277,0.089916,0.083838,0.00000,0.000000,...,0,0,0,0,0,0,3,0,259,328
1,0.089226,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.073487,0.12792,0.000000,...,0,0,0,0,0,0,0,0,393,523
2,0.036537,0.000000,0.0,0.046696,0.000000,0.000000,0.000000,0.090276,0.00000,0.000000,...,0,0,0,2,0,0,3,0,245,305
3,0.069680,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.086084,0.00000,0.000000,...,0,0,0,0,0,0,0,0,232,301
4,0.064426,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.106124,0.00000,0.000000,...,0,0,0,0,0,0,0,0,377,561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,0.068945,0.000000,0.0,0.088115,0.000000,0.000000,0.000000,0.056784,0.00000,0.000000,...,0,0,0,0,0,0,0,0,269,336
319,0.037198,0.000000,0.0,0.000000,0.133345,0.000000,0.000000,0.122547,0.00000,0.000000,...,0,0,0,0,0,0,1,0,218,283
320,0.000000,0.063843,0.0,0.069297,0.000000,0.000000,0.000000,0.044657,0.00000,0.000000,...,0,0,0,0,0,0,0,0,264,365
321,0.032724,0.000000,0.0,0.041823,0.000000,0.000000,0.000000,0.000000,0.00000,0.097336,...,0,0,0,0,0,0,0,0,234,320


In [20]:
cw = {0 :2.1, 1:1}
clf = LogisticRegression(random_state=42, class_weight=cw ,max_iter=1000).fit(X_train, y_train)
pedicitions = clf.predict(X_test)


In [21]:
predicted_category = ['m' if x==1 else 'f' for x in pedicitions]
test_example_id = test_df['test_example_id']

data = {
    'predicted_category': predicted_category,
    'test_example_id': test_example_id
}

results_df = pd.DataFrame(data= data, columns=['test_example_id','predicted_category'])
results_df

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f
...,...,...
318,318,m
319,319,m
320,320,f
321,321,m


In [22]:
results_df.to_csv('data/classification_results.csv')